### Learn his model and change the MSA layers to nn.transformerencoder, also, try and change any other layer that is in built in pytorch

### Model Properties from research paper

In [1]:
import torch
from torch import nn
import torch.nn.functional as F

encoder: stack N = 6 (identica layers)   
dmodel = 512   

decoder: stack N = 6 (identica layers)

h = 8  
dk,dv = 64   

hidden layer of ffn = 2048   




In [3]:
int(512/8)

64

In [4]:
emb = nn.Embedding(4,4)

In [5]:
emb

Embedding(4, 4)

In [79]:
zero =torch.randn(1,5)

In [80]:
probs = F.softmax(zero, dim=-1)

In [81]:
probs

tensor([[0.7955, 0.0705, 0.0674, 0.0073, 0.0593]])

In [74]:
zero_2 = torch.zeros(1,1)

In [83]:
zero_2, mn

(tensor([[0.]]), tensor([[0]]))

In [93]:
emb2 = nn.Embedding(65, 4)

In [94]:
val = torch.randint(4, (4,8))

In [96]:
val

tensor([[0, 2, 3, 3, 2, 2, 3, 1],
        [3, 2, 3, 0, 3, 3, 1, 1],
        [0, 0, 3, 1, 1, 1, 3, 3],
        [0, 0, 2, 0, 2, 2, 0, 0]])

In [82]:
mn = torch.multinomial(probs, num_samples=1)

In [77]:
torch.cat((zero_2, mn), dim=1)

tensor([[0., 2.]])

In [104]:
emb3 = nn.Embedding(32, 64)

In [105]:
emb3(torch.arange(32))

tensor([[-0.0548, -0.9601,  0.1128,  0.4235],
        [-1.1208, -0.3069,  0.3056,  0.9976],
        [ 0.1564,  0.2386, -1.0579, -0.2684],
        [-2.8524,  1.3360, -1.2760, -0.4714],
        [-0.2928,  0.8955, -0.6687, -1.0177],
        [ 0.4759, -0.0696,  0.4178,  0.6019],
        [ 0.1742,  1.5600,  1.1127,  1.3638],
        [-1.0526,  1.3567,  1.4657,  1.8995],
        [ 0.0511,  0.5668, -1.0124, -0.4460],
        [-0.5721,  0.4174, -0.1807,  0.1996],
        [ 0.8471,  0.4727, -0.2182,  0.9853],
        [ 1.5313, -2.3779, -0.2422,  1.1341],
        [ 0.3830, -0.3081,  0.7641,  1.3492],
        [ 0.3422,  0.4366, -1.5697, -0.3675],
        [ 0.0697,  0.1447, -0.3595, -1.4552],
        [-0.9424, -0.8859,  0.5325,  0.9046],
        [ 1.1276, -1.2876, -0.1206,  0.3778],
        [ 0.8339,  0.4186, -0.1903,  0.1530],
        [ 0.2991,  0.4172,  1.7093, -1.4071],
        [-0.4874, -0.3649,  0.2301,  0.6943],
        [ 0.1624, -0.0394, -1.4594, -0.9910],
        [ 1.1305, -0.4197, -1.1037

In [29]:
d_k = 64
d_v = 64
torch.manual_seed(44)
Q = torch.randn(1, d_k)
K = torch.randn(1, d_k)
V = torch.randn(1, d_v)

In [48]:
val = ((Q * K.T) / d_k**-0.5 ) @ V.permute(1,0)
self_attention = torch.softmax(val, dim=1)

In [10]:
n_embd = 64
head_size = 16
block_size =32
dropout = 0.0

In [9]:
torch.manual_seed(1337)
def get_vals(n_embd:int=64, head_size:int=16, batch_size:int=32):
    key = torch.randn(head_size, batch_size, n_embd)
    query = torch.randn(head_size, batch_size, n_embd)
    value = torch.randn(head_size, batch_size, n_embd)
    return key, query, value

In [7]:
torch.manual_seed(1337)
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size: int=16):
        super().__init__()
        self.key = torch.randn(n_embd, head_size)
        self.query = torch.randn(n_embd, head_size)
        self.value = torch.randn(n_embd, head_size)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

In [10]:
tril = torch.tril(torch.ones(block_size, block_size))

In [3]:
x = torch.randint(7, (3,4,5))

In [8]:
def manual_attn(q,k,v):
    B,T,C = k.shape
    wei = q @ k.transpose(-2,-1) * C**-0.5
    # is_casual
    wei = wei.masked_fill(tril[:T, :T] == 0, float('-inf')) # (B, T, T)
    # softmax
    wei = F.softmax(wei, dim=-1) # (B, T, T)
    # perform the weighted aggregation of the values
    # dropout
    wei = torch.dropout(wei, dropout, train=True)
    # v (B,T,C)
    out = wei @ v
    return out

In [11]:
torch.manual_seed(1337)
K, Q, V = get_vals()
man = manual_attn(Q,K,V)
# k[0, 0, :5], q[0, 0, :5], v[0,0, :5]

In [15]:
torch.isclose(F.scaled_dot_product_attention(
    query=Q, value=V, key=K, is_causal=True), manual_attn(Q, K,V))

tensor([[[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]],

        [[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]],

        [[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]],

In [13]:
torch.manual_seed(1337)
K, Q, V = get_vals()
auto = F.scaled_dot_product_attention(
    query=Q, value=V, key=K, is_causal=True)

In [14]:
auto[0,-3], man[0,-3]

(tensor([ 1.9849e-01, -4.1676e-04,  4.5094e-01,  1.0404e-01, -4.5471e-01,
          5.7427e-01,  2.3268e-01,  7.0840e-03,  8.1867e-02, -2.5917e-01,
          6.7024e-02, -3.2678e-02, -3.6319e-03,  4.2177e-01, -3.8028e-02,
         -6.6800e-02,  1.6193e-01, -3.3688e-02, -8.1977e-02, -9.1186e-02,
         -6.3470e-02, -6.5897e-02, -1.6090e-01,  2.0829e-01, -2.1415e-01,
          1.3640e-01, -2.6511e-01, -2.9911e-01, -1.1614e-01,  3.5740e-01,
         -1.5171e-01,  1.0641e-01, -5.2321e-02,  3.2750e-01,  1.5944e-01,
          4.4832e-01, -8.0178e-02,  1.1721e-01,  2.4602e-01, -4.6988e-01,
         -2.9049e-01, -2.3800e-02, -4.8197e-02, -3.0183e-01, -7.0186e-02,
          3.0087e-01, -3.7347e-01, -2.9339e-01, -7.2708e-02, -8.9672e-03,
         -2.2647e-02,  2.9277e-01,  1.9223e-01, -2.9739e-01, -1.5240e-01,
         -6.5217e-02, -4.3492e-01,  2.8869e-01, -1.7754e-01,  4.4380e-01,
         -1.4488e-01, -3.1673e-01, -4.4581e-02, -2.0972e-01]),
 tensor([ 0.2300, -0.0146,  0.4259,  0.1289, -0.4

### Building the transformer model with Pytorch in built layers

In [8]:
# hyperparameters
batch_size = 16  # how many independent sequences will we process in parallel?
block_size = 32  # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0

In [7]:

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
# encoder: take a string, output a list of integers
def encode(s): return [stoi[c] for c in s]
# decoder: take a list of integers, output a string
def decode(l): return ''.join([itos[i] for i in l])


# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))  # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading


def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)


class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        self.sa = nn.MultiheadAttention(embed_dim=n_embd, batch_first=True, dropout=dropout, num_heads=n_head)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        sa, _ = self.sa(query=self.ln1(x), key=self.ln1(x),
                        value=self.ln1(x), need_weights=False, is_causal=False)
        x = x + sa
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model


class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(
            *[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)  # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx)  # (B,T,C)
        pos_emb = self.position_embedding_table(
            torch.arange(T, device=device))  # (T,C)
        x = tok_emb + pos_emb  # (B,T,C)
        x = self.blocks(x)  # (B,T,C)
        x = self.ln_f(x)  # (B,T,C)
        logits = self.lm_head(x)  # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :]  # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx


model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(
            f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

0.210497 M parameters
step 0: train loss 4.3528, val loss 4.3497
step 4: train loss 3.7572, val loss 3.7612
step 8: train loss 3.5106, val loss 3.5048
step 9: train loss 3.4298, val loss 3.4780

tYf;xbRkRZrNdc.wf,fT O on,ebtK
baiPet bBbeA$ eXaSKgO-3 mMtc?gaea
hXbYV hthheNuhqhrthv.tbar dXlrhcaoe awccrPmRWf,fDsdaYzmzoeo X
YoR&$
mtof itihBiH&Vm W;KdilH,c
eg ireeYEenhciK;laW;HmtidroG tsSXUBeqWk  .eGhr. eWjbm!sel li F Ue u.t-huh3 onc mhy: Uup;MnissXhUwty. JlrBnUH koBfopeYeigvdEjMkm
tl wod motSkkleoaW-nso QhdVCeiib3s eTetm dE$riLETcee
hie$Fs  -LKKoAeh;TrH
he kahmntnruftef so;; ;QihW m:fEt,rey aleUo$tw,fMf Prr?d K So .trHK-NLbe! rtieb&i&a
 adsabWthehEghfsYBhuih KNtkrearaQey Rry tsmc&fy yEc!NMJkenE tnrPk3RJKry.tYoX 'WKh;RUehm sb a ;e hsN.-e?K;mfbRtwe teY.rtaJRY; sOzrbKiheG tn lrugy oIE.Ti mrc$  fsoyytsstDtcYs l.m 
 Eias zJtlKp ijTk.rseueXpPeAbe
emovetLKi?dCa?!c
eo os F.kmeeitKHKiei:  u
d hs xV  eRzKtALs:ytb$t'Cl  ;scPkn?iKYUto; dhRUe n,F,oxmg
e.hZthhhbeakwtO 
wHrc-Eu
 wwam k ! . D KUel nFc, er